In [ ]:
!pip install -q timm diffusers lpips transformers accelerate
!pip install -q torch torchvision pillow numpy scikit-learn matplotlib

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.cuda.amp import autocast
from torch.hub import load_state_dict_from_url
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from typing import Optional, Tuple, List, Dict, Union
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import timm
from diffusers import AutoencoderKL
import lpips
from google.colab import files
import io
from IPython.display import display, HTML

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if device.type == 'cuda':
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

In [ ]:
# DINO v2
def load_dinov2_extractor(model_name: str = 'vit_large_patch14_dinov2.lvd142m') -> nn.Module:
    print(f"Loading DINOv2 model: {model_name}")

    model = timm.create_model(
        model_name,
        pretrained=False,
        num_classes=0,
        global_pool='avg'
    )

    # 가중치 안가져오고 학습시킬 수는 없을까? 왜 자꾸 오류나지
    url = f"https://huggingface.co/timm/{model_name}/resolve/main/pytorch_model.bin"
    state_dict = load_state_dict_from_url(url, map_location='cpu')

    if 'norm.weight' in state_dict and 'fc_norm.weight' not in state_dict:
        state_dict['fc_norm.weight'] = state_dict.pop('norm.weight')
        state_dict['fc_norm.bias'] = state_dict.pop('norm.bias')
        print("State_dict keys 'norm' were successfully renamed to 'fc_norm'")

    model.load_state_dict(state_dict)

    model.eval()
    model = model.to(device)

    for param in model.parameters():
        param.requires_grad = False

    print("✓ DINOv2 extractor loaded successfully")
    return model

In [ ]:
# AutoEncoder
def load_sd_vae(model_id: str = "stabilityai/stable-diffusion-2-1") -> AutoencoderKL:
    print(f"Loading Stable Diffusion VAE from: {model_id}...")

    vae = AutoencoderKL.from_pretrained(
        model_id,
        subfolder="vae",
        torch_dtype=torch.float16 if device.type == 'cuda' else torch.float32
    )

    vae.eval()
    vae = vae.to(device)

    for param in vae.parameters():
        param.requires_grad = False

    print("✓ VAE loaded successfully")
    return vae

In [ ]:
# RIGID
# S(x) = 1{sim(f(x), f(x + λ·δ)) ≤ ε}; δ ~ N(0,I)
def detect_with_rigid(
    image_tensor: torch.Tensor,             # B, C, H, W 형태의 정규화된 입력 이미지 텐서
    dinov2_feature_extractor: nn.Module,    # 특징 추출기
    noise_level: float = 0.05               # 이미지에 추가할 가우시안 노이즈의 표준편차 (λ)
) -> torch.Tensor:

    with torch.no_grad():
        # 원본 이미지에서 특징 추출
        original_features = dinov2_feature_extractor(image_tensor)

        # 가우시안 노이즈 생성 (δ ~ N(0,I))
        noise = torch.randn_like(image_tensor) * noise_level

        # 노이즈가 추가된 이미지 생성
        noisy_image = image_tensor + noise

        # 노이즈 이미지에서 특징 추출
        noisy_features = dinov2_feature_extractor(noisy_image)

        # 코사인 유사도 계산
        # F.cosine_similarity는 배치 차원을 유지하면서 계산
        cosine_similarity = F.cosine_similarity(
            original_features,
            noisy_features,
            dim=1
        )

        # 배치 크기가 1보다 큰 경우 차원 조정
        if cosine_similarity.dim() == 0:
            cosine_similarity = cosine_similarity.unsqueeze(0)

    return cosine_similarity

# 원본과 노이즈 이미지 특징 간의 코사인 유사도 점수
# 낮은 값일수록 AI 생성 이미지일 가능성이 높음

In [ ]:
# AEROBLADE
# ∆AE(x) = d(x, D(E(x))) where d is LPIPS distance
def detect_with_aeroblade(
    image_tensor: torch.Tensor,             # B, C, H, W 형태의 입력 이미지 텐서 [-1, 1]
    diffusion_autoencoder: AutoencoderKL,   # Stable Diffusion의 사전 학습된 VAE
    lpips_loss_fn: lpips.LPIPS              # LPIPS 손실 함수
) -> torch.Tensor:

    with torch.no_grad():
        # VAE 인코더를 통해 latent 표현 획득
        if device.type == 'cuda' and diffusion_autoencoder.dtype == torch.float16:
            with autocast():
                latent_dist = diffusion_autoencoder.encode(image_tensor)
                latent = latent_dist.latent_dist.sample()

                # VAE 디코더를 통해 이미지 복원
                reconstructed = diffusion_autoencoder.decode(latent).sample
        else:
            latent_dist = diffusion_autoencoder.encode(image_tensor)
            latent = latent_dist.latent_dist.sample()
            reconstructed = diffusion_autoencoder.decode(latent).sample

        # LPIPS 복원 오류 계산 [-1, 1]
        lpips_distance = lpips_loss_fn(image_tensor, reconstructed)

        # 배치 차원에서 평균 계산 (spatial dimensions에 대해)
        lpips_score = lpips_distance.squeeze()

        # 스칼라로 변환 (배치 크기가 1인 경우)
        if lpips_score.dim() == 0:
            lpips_score = lpips_score.unsqueeze(0)

    return lpips_score

# 원본과 복원 이미지 간의 LPIPS 복원 오류 점수
# 낮은 값일수록 AI 생성 이미지일 가능성이 높음

In [ ]:
print("<모델 초기화>")
dinov2_model = load_dinov2_extractor()
sd_vae = load_sd_vae()
lpips_fn = lpips.LPIPS(net='vgg').to(device)
lpips_fn.eval()
print("<모델 로드 완료>")

In [ ]:
# 이미지 전처리
class ImagePreprocessor:
    def __init__(self, target_size: int = 518): # DINOv2 입력을 위한 목표 이미지 크기
        self.target_size = target_size

        # DINOv2용 변환 (ImageNet 정규화)
        self.dinov2_transform = transforms.Compose([
            transforms.Resize((target_size, target_size), interpolation=transforms.InterpolationMode.BICUBIC),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

        # VAE용 변환 ([-1, 1] 정규화)
        self.vae_transform = transforms.Compose([
            transforms.Resize((512, 512), interpolation=transforms.InterpolationMode.BICUBIC),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
        ])

    def process_for_dinov2(self, image: Image.Image) -> torch.Tensor:
        return self.dinov2_transform(image).unsqueeze(0).to(device)

    def process_for_vae(self, image: Image.Image) -> torch.Tensor:
        return self.vae_transform(image).unsqueeze(0).to(device)

In [ ]:
preprocessor = ImagePreprocessor()

In [ ]:
def visualize_results(results: List[Dict]):
    num_images = len(results)
    fig, axes = plt.subplots(num_images, 3, figsize=(15, 5 * num_images), squeeze=False)

    for idx, result in enumerate(results):
        axes[idx, 0].imshow(result['image'])
        axes[idx, 0].set_title(f"Input: {result['name']}", fontsize=12, fontweight='bold')
        axes[idx, 0].axis('off')

        rigid_score = result['rigid_score']
        rigid_color = 'green' if rigid_score > 0.998 else 'red'
        axes[idx, 1].barh(['RIGID'], [rigid_score], color=rigid_color, alpha=0.7)
        axes[idx, 1].set_xlim(0, 1)
        axes[idx, 1].set_xlabel('Cosine Similarity (Higher is Real)')
        axes[idx, 1].set_title(f"RIGID Score: {rigid_score:.4f}")
        axes[idx, 1].axvline(x=0.998, color='black', linestyle='--', alpha=0.5)

        aeroblade_score = result['aeroblade_score']
        aero_color = 'green' if aeroblade_score > 0.085 else 'red'
        axes[idx, 2].barh(['AEROBLADE'], [aeroblade_score], color=aero_color, alpha=0.7)
        axes[idx, 2].set_xlim(0, 0.5)
        axes[idx, 2].set_xlabel('LPIPS Distance (Higher is Real)')
        axes[idx, 2].set_title(f"AEROBLADE Score: {aeroblade_score:.4f}")
        axes[idx, 2].axvline(x=0.085, color='black', linestyle='--', alpha=0.5)

    plt.suptitle('AI-Generated Image Detection Results', fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.show()

In [ ]:
def run_detection_demo():
    print("🔍 AI 생성 이미지 탐지 데모")
    print("\n📁 이미지를 업로드해주세요.")
    print("  1. 실제 이미지 1개")
    print("  2. AI 생성 이미지 1개")
    print("\n")

    uploaded = files.upload()

    if len(uploaded) != 2:
        print(f"❌ 오류: 2개의 이미지를 업로드해야 합니다.")
        return

    images, image_names = [], []
    for filename, content in uploaded.items():
        try:
            image = Image.open(io.BytesIO(content)).convert('RGB')
            images.append(image)
            image_names.append(filename)
        except Exception as e:
            print(f"❌ 오류: {filename} 파일을 열 수 없습니다. {str(e)}")
            return

    print(f"\n✓ 이미지 로드 완료: {image_names}")

    results = []
    for image, name in zip(images, image_names):
        print(f"\n\n분석 중: {name}\n")

        dinov2_input = preprocessor.process_for_dinov2(image)
        vae_input = preprocessor.process_for_vae(image)

        rigid_score = detect_with_rigid(dinov2_input, dinov2_model).item()
        aeroblade_score = detect_with_aeroblade(vae_input, sd_vae, lpips_fn).item()

        results.append({
            'name': name, 'image': image,
            'rigid_score': rigid_score, 'aeroblade_score': aeroblade_score
        })

        print(f"📊 RIGID Score: {rigid_score:.4f}")
        print(f"   → {'🟢 실제 이미지' if rigid_score > 0.998 else '🔴 AI 생성 의심'}")
        print(f"\n📊 AEROBLADE Score: {aeroblade_score:.4f}")
        print(f"   → {'🟢 실제 이미지' if aeroblade_score > 0.085 else '🔴 AI 생성 의심'}")

    print("\n\n")
    visualize_results(results)
    return results

#  **사용방법**
- 아래 코드 셀을 실행한 후 나타나는 **'파일 선택'** 버튼을 클릭하세요
- **정확히 2개의 이미지**를 업로드해야 합니다.
  1. 첫 번째 이미지: **실제(Real) 이미지**
  2. 두 번째 이미지: **AI 생성(Fake) 이미지**
- 지원 형식: JPG, PNG, WEBP
- 권장 이미지 크기: 512x512 픽셀 이상

In [ ]:
run_detection_demo()